In [7]:
import math
import torch
import gpytorch
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
path = r"C:\Users\mikep\Desktop\project\data\final"

Xtrain = torch.from_numpy(np.genfromtxt(path+"\Xtrain.csv", delimiter=",", skip_header=1))
Ytrain = torch.from_numpy(np.genfromtxt(path+"\Ytrain.csv", delimiter=",")).t()

Xtest = torch.from_numpy(np.genfromtxt(path+"\Xtest.csv", delimiter=",", skip_header=1))
Ytest = torch.from_numpy(np.genfromtxt(path+"\Ytest.csv", delimiter=",")).t()

print(Xtrain.size(), Ytrain.size(), Xtest.size(), Ytest.size())
Xtrain.type()

torch.Size([20, 2]) torch.Size([20]) torch.Size([10, 2]) torch.Size([10])


'torch.DoubleTensor'

In [ ]:
#Xtrain = Xtrain.type(torch.float32)
#Ytrain = Ytrain.type(torch.float32)
#Xtest = Xtest.type(torch.float32)
#Ytest = Ytest.type(torch.float32)

In [ ]:
#normalization

In [3]:
class GPRegression(gpytorch.models.ExactGP):
    def __init__(self, Xtrain, Ytrain, likelihood):
        super(GPRegression, self).__init__(Xtrain, Ytrain, likelihood)
        self.mean_module = gpytorch.means.ZeroMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(ard_num_dims=2))
    
    def forward(self, x):
        mean = self.mean_module(x)
        covar = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean, covar)

In [4]:
# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = GPRegression(Xtrain, Ytrain, likelihood)

In [5]:
# this is for running the notebook in our testing framework
import os
smoke_test = ('CI' in os.environ)
training_iter = 2 if smoke_test else 100

# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

In [6]:
for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(Xtrain)
    # Calc loss and backprop gradients
    loss = -mll(output, Ytrain)
    loss.backward()
    print("Iter %d/%d - Loss: %.3f   lengthscales: %.3f, %.3f   noise: %.3f" % (
        i + 1, training_iter, loss.item(),
        model.covar_module.base_kernel.lengthscale.squeeze()[0],
        model.covar_module.base_kernel.lengthscale.squeeze()[1],
        model.likelihood.noise.item()
    ))
    optimizer.step()

C:\Users\mikep\anaconda3\lib\site-packages\gpytorch\lazy\triangular_lazy_tensor.py:130: UserWarning: torch.triangular_solve is deprecated in favor of torch.linalg.solve_triangularand will be removed in a future PyTorch release.
torch.linalg.solve_triangular has its arguments reversed and does not return a copy of one of the inputs.
X = torch.triangular_solve(B, A).solution
should be replaced with
X = torch.linalg.solve_triangular(A, B). (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\BatchLinearAlgebra.cpp:1672.)
  res = torch.triangular_solve(right_tensor, self.evaluate(), upper=self.upper).solution


Iter 1/100 - Loss: 1520.038   lengthscales: 0.693, 0.693   noise: 0.693
Iter 2/100 - Loss: 1415.505   lengthscales: 0.693, 0.693   noise: 0.744
Iter 3/100 - Loss: 1320.631   lengthscales: 0.693, 0.693   noise: 0.798
Iter 4/100 - Loss: 1234.616   lengthscales: 0.693, 0.693   noise: 0.854
Iter 5/100 - Loss: 1156.691   lengthscales: 0.693, 0.693   noise: 0.911
Iter 6/100 - Loss: 1086.125   lengthscales: 0.693, 0.693   noise: 0.971
Iter 7/100 - Loss: 1022.231   lengthscales: 0.693, 0.693   noise: 1.031
Iter 8/100 - Loss: 964.368   lengthscales: 0.693, 0.693   noise: 1.093
Iter 9/100 - Loss: 911.942   lengthscales: 0.693, 0.693   noise: 1.156
Iter 10/100 - Loss: 864.411   lengthscales: 0.693, 0.693   noise: 1.220
Iter 11/100 - Loss: 821.278   lengthscales: 0.693, 0.693   noise: 1.284
Iter 12/100 - Loss: 782.093   lengthscales: 0.693, 0.693   noise: 1.349
Iter 13/100 - Loss: 746.450   lengthscales: 0.693, 0.693   noise: 1.413
Iter 14/100 - Loss: 713.984   lengthscales: 0.693, 0.693   noise: 

In [ ]:
# Get into evaluation (predictive posterior) mode
model.eval()
likelihood.eval()

# Make predictions by feeding model through likelihood
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    observed_pred = likelihood(model(Xtest))

In [ ]:
print(Xtrain.size(), Xtest.size())

In [ ]:
#plotting fit

with torch.no_grad():
    # Initialize plot
    f, ax = plt.subplots(1, 1, figsize=(4, 4))
    # Get upper and lower confidence bounds
    lower, upper = observed_pred.confidence_region()
    # Plot training data as black stars
    ax.plot(Xtrain[:, 0].numpy(), Ytrain.numpy(), '*')
    # Plot predictive means as blue line
    ax.plot(Xtest[:, 0].numpy(), observed_pred.mean.numpy(), 'b')
    # Shade between the lower and upper confidence bounds
    ax.fill_between(Xtest[:, 0].numpy(), lower.numpy(), upper.numpy(), alpha=0.5)
    #ax.set_ylim([-3, 3])
    ax.legend(['Observed Data', 'Mean', 'Confidence'])

In [ ]:
# Set into eval mode
model.eval()
likelihood.eval()

# Initialize plots
fig, ax = plt.subplots(2, 3, figsize=(14, 10))

# Test points
n1, n2 = 50, 50
xv, yv = torch.meshgrid([torch.linspace(0, 1, n1), torch.linspace(0, 1, n2)])
f, dfx, dfy = franke(xv, yv)

# Make predictions
with torch.no_grad(), gpytorch.settings.fast_computations(log_prob=False, covar_root_decomposition=False):
    test_x = torch.stack([xv.reshape(n1*n2, 1), yv.reshape(n1*n2, 1)], -1).squeeze(1)
    predictions = likelihood(model(test_x))
    mean = predictions.mean

extent = (xv.min(), xv.max(), yv.max(), yv.min())
ax[0, 0].imshow(f, extent=extent, cmap=cm.jet)
ax[0, 0].set_title('True values')
ax[0, 1].imshow(dfx, extent=extent, cmap=cm.jet)
ax[0, 1].set_title('True x-derivatives')
ax[0, 2].imshow(dfy, extent=extent, cmap=cm.jet)
ax[0, 2].set_title('True y-derivatives')

ax[1, 0].imshow(mean[:, 0].detach().numpy().reshape(n1, n2), extent=extent, cmap=cm.jet)
ax[1, 0].set_title('Predicted values')
ax[1, 1].imshow(mean[:, 1].detach().numpy().reshape(n1, n2), extent=extent, cmap=cm.jet)
ax[1, 1].set_title('Predicted x-derivatives')
ax[1, 2].imshow(mean[:, 2].detach().numpy().reshape(n1, n2), extent=extent, cmap=cm.jet)
ax[1, 2].set_title('Predicted y-derivatives')

None